In [1]:
import pandas as pd
import numpy as np

The idea is to caluclate number and percentage of genera or ASVs that are in 4 groups:  
1) shared across countries, means that there is at least one genus in each country  
2) specific to country - at least one genus in single country, but not in another  
3) specific to region - at least one genus in single region, but now in another  
4) site-specific - at least one genus in a single site only  

# DATA

In [2]:
genus_16s = "02_core_16S_site_GENUS.tsv"
genus_ITS = "05_core_ITS_site_GENUS.tsv"
asv_16s = "02_core_16S_site_ASV.tsv"
asv_ITS = "05_core_ITS_site_ASV.tsv"

In [3]:
pan = ['11']
country_specific = ['10','01']
region_specific = ['100','010','001']
site_specific = ['10000000', '01000000','00100000','00010000','00001000','00000100','00000010','00000001']


# One file example

In [4]:
df = pd.read_csv(genus_ITS, sep="\t", header=0)
df.head()

,Row.names,AMOS,ESSI,FLOR1,FORE,FP2,STET,STFE,Santiago,count,genus_abd
0,Fungi_Ascomycota_Archaeorhizomycetes_Archaeorh...,1,1,1,1,1,1,1,1,8,5803
1,Fungi_Ascomycota_Archaeorhizomycetes_unid_unid...,1,1,0,1,0,0,0,0,3,167
2,Fungi_Ascomycota_Dothideomycetes_Acrospermales...,0,0,0,0,0,1,0,0,1,21
3,Fungi_Ascomycota_Dothideomycetes_Botryosphaeri...,0,0,0,0,0,0,0,1,1,38
4,Fungi_Ascomycota_Dothideomycetes_Botryosphaeri...,0,0,1,0,1,0,0,1,3,1424


In [5]:
df = pd.read_csv(asv_ITS, sep="\t", header=0)
print(df.shape)
# Removing genera with no occurences
df['Site'] = df.apply(lambda x: str(x['AMOS'])+str(x['STFE'])+str(x['STET'])+str(x['ESSI'])+str(x['FORE'])+str(x['FLOR1'])+str(x['FP2'])+str(x['Santiago']), axis=1)
df = df[df['Site'] != '00000000'].reset_index(drop=True)
print(df.shape)
# Patterns for countries and regions
df['Canada'] = df.apply(lambda x: max(x['AMOS'],x['STFE'],x['STET'],x['ESSI'],x['FORE']), axis=1)
df['Mexico'] = df.apply(lambda x: max(x['FLOR1'],x['FP2'],x['Santiago']), axis=1)
df['Country'] = df.apply(lambda x: str(x['Canada'])+str(x['Mexico']), axis=1)
df['Boreal'] = df.apply(lambda x: max(x['AMOS'],x['STFE']), axis=1)
df['ColdTemperate'] = df.apply(lambda x: max(x['STET'],x['ESSI'],x['FORE']), axis=1)
df['WarmTemperate'] = df.apply(lambda x: max(x['FLOR1'],x['FP2'],x['Santiago']), axis=1)
df['Region'] = df.apply(lambda x: str(x['Boreal'])+str(x['ColdTemperate'])+str(x['WarmTemperate']), axis=1)
df.head()
df.columns

(7320, 18)
(5024, 19)


Index(['Row.names', 'AMOS', 'ESSI', 'FLOR1', 'FORE', 'FP2', 'STET', 'STFE',
       'Santiago', 'Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus',
       'Species', 'count', 'asv_abd', 'Site', 'Canada', 'Mexico', 'Country',
       'Boreal', 'ColdTemperate', 'WarmTemperate', 'Region'],
      dtype='object')

In [6]:

df['Pan-country_specific_group1'] = df['Country'].apply(lambda x: 1 if x in pan else 0)
df['Country-specific_specific_group2'] = df['Country'].apply(lambda x: 1 if x in country_specific else 0)
df['Canada_spec_group2'] = df['Country'].apply(lambda x: 1 if x in ['10'] else 0)
df['Mexico_spec_group2'] = df['Country'].apply(lambda x: 1 if x in ['01'] else 0)

df['Region-specific_specific_group3'] = df['Region'].apply(lambda x: 1 if x in region_specific else 0)
df['Boreal_spec_group3'] = df['Region'].apply(lambda x: 1 if x in ['100'] else 0)
df['Cold-temperate_spec_group3'] = df['Region'].apply(lambda x: 1 if x in ['010'] else 0)
df['Warm-temperate_spec_group3'] = df['Region'].apply(lambda x: 1 if x in ['001'] else 0)

df['Site-specific_specific_group4'] = df['Site'].apply(lambda x: 1 if x in site_specific else 0)
df['AMOS_spec_group4'] = df['Site'].apply(lambda x: 1 if x in ['10000000'] else 0)
df['STFE_spec_group4'] = df['Site'].apply(lambda x: 1 if x in ['01000000'] else 0)
df['STET_spec_group4'] = df['Site'].apply(lambda x: 1 if x in ['00100000'] else 0)
df['ESSI_spec_group4'] = df['Site'].apply(lambda x: 1 if x in ['00010000'] else 0)
df['FORE_spec_group4'] = df['Site'].apply(lambda x: 1 if x in ['00001000'] else 0)
df['FLOR1_spec_group4'] = df['Site'].apply(lambda x: 1 if x in ['00000100'] else 0)
df['FP2_spec_group4'] = df['Site'].apply(lambda x: 1 if x in ['00000010'] else 0)
df['Santiago_spec_group4'] = df['Site'].apply(lambda x: 1 if x in ['00000001'] else 0)

df.head()

,Row.names,AMOS,ESSI,FLOR1,FORE,FP2,STET,STFE,Santiago,Domain,...,Warm-temperate_spec_group3,Site-specific_specific_group4,AMOS_spec_group4,STFE_spec_group4,STET_spec_group4,ESSI_spec_group4,FORE_spec_group4,FLOR1_spec_group4,FP2_spec_group4,Santiago_spec_group4
0,3b3ca36e7e48dfeb9651339fb5e89db6,0,1,0,0,0,1,0,0,NaN,...,0,0,0,0,0,0,0,0,0,0
1,dbdea2d4f96c7f353b406b225ea921d1,1,0,0,0,0,0,0,0,NaN,...,0,1,1,0,0,0,0,0,0,0
2,2ab2a8d9866b45cbd9ee58d672b3056b,1,0,0,0,0,0,0,0,NaN,...,0,1,1,0,0,0,0,0,0,0
3,8cdeb43b63256c27cafbe41d77f82b51,0,1,0,0,0,0,0,0,NaN,...,0,1,0,0,0,1,0,0,0,0
4,b738dfc3e5294595feb0c0abda1179d8,0,0,1,0,1,0,0,0,NaN,...,1,0,0,0,0,0,0,0,0,0


In [7]:
df.columns

Index(['Row.names', 'AMOS', 'ESSI', 'FLOR1', 'FORE', 'FP2', 'STET', 'STFE',
       'Santiago', 'Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus',
       'Species', 'count', 'asv_abd', 'Site', 'Canada', 'Mexico', 'Country',
       'Boreal', 'ColdTemperate', 'WarmTemperate', 'Region',
       'Pan-country_specific_group1', 'Country-specific_specific_group2',
       'Canada_spec_group2', 'Mexico_spec_group2',
       'Region-specific_specific_group3', 'Boreal_spec_group3',
       'Cold-temperate_spec_group3', 'Warm-temperate_spec_group3',
       'Site-specific_specific_group4', 'AMOS_spec_group4', 'STFE_spec_group4',
       'STET_spec_group4', 'ESSI_spec_group4', 'FORE_spec_group4',
       'FLOR1_spec_group4', 'FP2_spec_group4', 'Santiago_spec_group4'],
      dtype='object')

In [8]:
potential_columns_to_drop = ['AMOS','ESSI','FLOR1','FORE','FP2','STET','STFE','Santiago','count','genus_abd', 'asv_abd', 'Site', 'Canada', 'Mexico', 'Country',
       'Boreal', 'ColdTemperate', 'WarmTemperate', 'Region','Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus','Species']
present_columns_to_drop = [ele for ele in potential_columns_to_drop if ele in df.columns]
dsub = df.drop(columns=present_columns_to_drop).reset_index(drop=True)
dstack = dsub.set_index('Row.names').stack(
).reset_index().rename(columns={'level_1':'group',0:'presence'})
dg = dstack.groupby('group').agg(n = ('presence','sum'),
                            total = ('presence', 'count')).reset_index()

dg['percent'] = dg.apply(lambda x: (x['n']/x['total'])*100, axis=1)
dg['label'] = dg['group'].apply(lambda x: x.split('_')[0])
dg['group_id'] = dg['group'].apply(lambda x: x.split('_')[2])
dg['color_group'] = dg['group'].apply(lambda x: x.split('_')[1])

dg

,group,n,total,percent,label,group_id,color_group
0,AMOS_spec_group4,308,5024,6.130573,AMOS,group4,spec
1,Boreal_spec_group3,594,5024,11.823248,Boreal,group3,spec
2,Canada_spec_group2,2472,5024,49.203822,Canada,group2,spec
3,Cold-temperate_spec_group3,1567,5024,31.190287,Cold-temperate,group3,spec
4,Country-specific_specific_group2,4122,5024,82.046178,Country-specific,group2,specific
5,ESSI_spec_group4,435,5024,8.658439,ESSI,group4,spec
6,FLOR1_spec_group4,247,5024,4.916401,FLOR1,group4,spec
7,FORE_spec_group4,357,5024,7.105892,FORE,group4,spec
8,FP2_spec_group4,297,5024,5.911624,FP2,group4,spec
9,Mexico_spec_group2,1650,5024,32.842357,Mexico,group2,spec


# Loop

In [9]:
genus_16s = "02_core_16S_site_GENUS.tsv"
genus_ITS = "05_core_ITS_site_GENUS.tsv"
asv_16s = "02_core_16S_site_ASV.tsv"
asv_ITS = "05_core_ITS_site_ASV.tsv"

In [10]:
pan = ['11']
country_specific = ['10','01']
pan_region = ['111']
region_specific = ['100','010','001']
pan_site = ['11111111']
site_specific = ['10000000', '01000000','00100000','00010000','00001000','00000100','00000010','00000001']


In [11]:
for filename in [genus_16s, genus_ITS, asv_16s, asv_ITS]:
    print(filename)
    df = pd.read_csv(filename, sep="\t", header=0)
    print(df.shape)
    # Removing genera with no occurences
    df['Site'] = df.apply(lambda x: str(x['AMOS'])+str(x['STFE'])+str(x['STET'])+str(x['ESSI'])+str(x['FORE'])+str(x['FLOR1'])+str(x['FP2'])+str(x['Santiago']), axis=1)
    df = df[df['Site'] != '00000000'].reset_index(drop=True)
    print(df.shape)
    # Patterns for countries and regions
    df['Canada'] = df.apply(lambda x: max(x['AMOS'],x['STFE'],x['STET'],x['ESSI'],x['FORE']), axis=1)
    df['Mexico'] = df.apply(lambda x: max(x['FLOR1'],x['FP2'],x['Santiago']), axis=1)
    df['Country'] = df.apply(lambda x: str(x['Canada'])+str(x['Mexico']), axis=1)
    df['Boreal'] = df.apply(lambda x: max(x['AMOS'],x['STFE']), axis=1)
    df['ColdTemperate'] = df.apply(lambda x: max(x['STET'],x['ESSI'],x['FORE']), axis=1)
    df['WarmTemperate'] = df.apply(lambda x: max(x['FLOR1'],x['FP2'],x['Santiago']), axis=1)
    df['Region'] = df.apply(lambda x: str(x['Boreal'])+str(x['ColdTemperate'])+str(x['WarmTemperate']), axis=1)

    df['Pan-population_shared_Population'] = df['Country'].apply(lambda x: 1 if x in pan else 0)
    df['Country-specific_unique_Population'] = df['Country'].apply(lambda x: 1 if x in country_specific else 0)
    df['Canada_spec_Population'] = df['Country'].apply(lambda x: 1 if x in ['10'] else 0)
    df['Mexico_spec_Population'] = df['Country'].apply(lambda x: 1 if x in ['01'] else 0)

    df['Pan-region_shared_Region'] = df['Region'].apply(lambda x: 1 if x in ['111'] else 0)
    df['Region-specific_unique_Region'] = df['Region'].apply(lambda x: 1 if x in region_specific else 0)
    df['Boreal_spec_Region'] = df['Region'].apply(lambda x: 1 if x in ['100'] else 0)
    df['Cold-temperate_spec_Region'] = df['Region'].apply(lambda x: 1 if x in ['010'] else 0)
    df['Warm-temperate_spec_Region'] = df['Region'].apply(lambda x: 1 if x in ['001'] else 0)

    df['Pan-site_shared_Site'] = df['Site'].apply(lambda x: 1 if x in ['11111111'] else 0)
    df['Site-specific_unique_Site'] = df['Site'].apply(lambda x: 1 if x in site_specific else 0)
    df['AMOS_spec_Site'] = df['Site'].apply(lambda x: 1 if x in ['10000000'] else 0)
    df['STFE_spec_Site'] = df['Site'].apply(lambda x: 1 if x in ['01000000'] else 0)
    df['STET_spec_Site'] = df['Site'].apply(lambda x: 1 if x in ['00100000'] else 0)
    df['ESSI_spec_Site'] = df['Site'].apply(lambda x: 1 if x in ['00010000'] else 0)
    df['FORE_spec_Site'] = df['Site'].apply(lambda x: 1 if x in ['00001000'] else 0)
    df['FLOR1_spec_Site'] = df['Site'].apply(lambda x: 1 if x in ['00000100'] else 0)
    df['FP2_spec_Site'] = df['Site'].apply(lambda x: 1 if x in ['00000010'] else 0)
    df['Santiago_spec_Site'] = df['Site'].apply(lambda x: 1 if x in ['00000001'] else 0)

    potential_columns_to_drop = ['AMOS','ESSI','FLOR1','FORE','FP2','STET','STFE','Santiago','count','genus_abd', 'asv_abd', 'Site', 'Canada', 'Mexico', 'Country',
           'Boreal', 'ColdTemperate', 'WarmTemperate', 'Region','Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus','Species']
    present_columns_to_drop = [ele for ele in potential_columns_to_drop if ele in df.columns]
    dsub = df.drop(columns=present_columns_to_drop).reset_index(drop=True)
    dstack = dsub.set_index('Row.names').stack(
    ).reset_index().rename(columns={'level_1':'group',0:'presence'})
    dg = dstack.groupby('group').agg(n = ('presence','sum'),
                                total = ('presence', 'count')).reset_index()
    dg['percent'] = dg.apply(lambda x: (x['n']/x['total'])*100, axis=1)
    dg['label'] = dg['group'].apply(lambda x: x.split('_')[0])
    dg['group_id'] = dg['group'].apply(lambda x: x.split('_')[2])
    dg['color_group'] = dg['group'].apply(lambda x: x.split('_')[1])

    # "02_core_16S_site_GENUS.tsv"
    fileID = filename.replace("02_core_","").replace("05_core_","").replace(".tsv","")
    dg.to_csv("11_get_shared_groups_"+fileID+".tsv", sep="\t", header=True, index=False)

02_core_16S_site_GENUS.tsv
(724, 11)
(653, 12)
05_core_ITS_site_GENUS.tsv
(676, 11)
(603, 12)
02_core_16S_site_ASV.tsv
(15486, 18)
(11517, 19)
05_core_ITS_site_ASV.tsv
(7320, 18)
(5024, 19)
